In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:
# Carregar dados
file_path = 'C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx'
data = pd.read_excel(file_path, sheet_name='Planilha1')

In [3]:
# Processar dados
data['Data Referência'] = pd.to_datetime(data['Data Referência'])
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()

X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']

In [4]:
# Divisão dos dados respeitando ordem temporal
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

In [5]:
# Parâmetros para seleção de features
k_values = [5, 10, 15]
percentile_values = [10, 20, 30]
melhor_r2 = float('-inf')
melhor_X_train = None
melhor_X_test = None
melhores_features = None

In [6]:
# FEATURE SELECTION

for k in k_values:
    for p in percentile_values:
        skb = SelectKBest(score_func=f_regression, k=min(k, X_train.shape[1]))
        sp = SelectPercentile(score_func=f_regression, percentile=p)
        
        X_train_kbest = skb.fit_transform(X_train, y_train)
        X_test_kbest = skb.transform(X_test)
        selected_features_kbest = X_train.columns[skb.get_support()]
        
        X_train_percentile = sp.fit_transform(X_train, y_train)
        X_test_percentile = sp.transform(X_test)
        selected_features_percentile = X_train.columns[sp.get_support()]
        
        for X_train_selected, X_test_selected, selected_features in [
            (X_train_kbest, X_test_kbest, selected_features_kbest), 
            (X_train_percentile, X_test_percentile, selected_features_percentile)]:
            
            model = LinearRegression()
            model.fit(X_train_selected, y_train)
            y_pred = model.predict(X_test_selected)
            r2 = r2_score(y_test, y_pred)
            
            if r2 > melhor_r2:
                melhor_r2 = r2
                melhor_X_train = X_train_selected
                melhor_X_test = X_test_selected
                melhores_features = selected_features

print(f"Melhores features selecionadas: {list(melhores_features)}")

Melhores features selecionadas: ['Valor IPCA', 'IPCA Mês -12', 'IPCA Mês -11', 'IPCA Mês -10', 'IPCA Mês -9', 'IPCA Mês -8', 'IPCA Mês -7', 'IPCA Mês -6', 'IPCA Mês -5', 'IPCA Mês -4', 'IPCA Mês -3', 'IPCA Mês -2', 'DOLAR Mês -11', 'DOLAR Mês -10', 'DOLAR Mês -9', 'DOLAR Mês -8', 'DOLAR Mês -7', 'DOLAR Mês -6', 'DOLAR Mês -5', 'DOLAR Mês -4', 'DOLAR Mês -3', 'DOLAR Mês -2']


In [7]:
# Modelos a serem testados
modelos = {
    "Regressão Linear": LinearRegression(),
    "SVR": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor()
}

In [8]:
# GridSearch para cada modelo
for nome, modelo in modelos.items():
    parametros = {}
    if isinstance(modelo, SVR):
        parametros = {'C': [0.1, 1, 10], 'epsilon': [0.001, 0.01, 0.1], 'kernel': ['linear', 'rbf']}
    elif isinstance(modelo, DecisionTreeRegressor):
        parametros = {'max_depth': [3, 5, 10], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
    elif isinstance(modelo, RandomForestRegressor):
        parametros = {'n_estimators': [100, 200], 'max_depth': [None, 10], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}
    elif isinstance(modelo, GradientBoostingRegressor):
        parametros = {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1], 'max_depth': [3, 5]}
    
    grid_search = GridSearchCV(modelo, parametros, cv=5, scoring= 'r2', refit=True)
    grid_search.fit(melhor_X_train, y_train)
    
    melhor_modelo = grid_search.best_estimator_
    y_pred = melhor_modelo.predict(melhor_X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"Modelo: {nome}")
    print(f"Melhores parâmetros: {grid_search.best_params_}")
    print(f"MSE: {mse:.4f}")
    print(f"R²: {r2:.4f}")
    print("="*50)

Modelo: Regressão Linear
Melhores parâmetros: {}
MSE: 0.0001
R²: 0.9654
Modelo: SVR
Melhores parâmetros: {'C': 10, 'epsilon': 0.001, 'kernel': 'linear'}
MSE: 0.0001
R²: 0.9532
Modelo: Decision Tree
Melhores parâmetros: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2}
MSE: 0.0254
R²: -10.2729
Modelo: Random Forest
Melhores parâmetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
MSE: 0.0173
R²: -6.6617
Modelo: Gradient Boosting
Melhores parâmetros: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
MSE: 0.0097
R²: -3.2911


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

# Carregar dados
data = pd.read_excel('C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx', sheet_name='Planilha1')
data['Data Referência'] = pd.to_datetime(data['Data Referência'])
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()

X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']

# Divisão dos dados
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Definir hiperparâmetros para GridSearch
param_grid_mlp = {
    'hidden_layer_sizes': [(64,), (128,), (256,), (64, 32), (128, 64, 32)],
    'max_iter': [1000, 2000, 5000],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'activation': ['relu', 'tanh']
}

# Criar modelo e aplicar GridSearch
mlp = MLPRegressor(random_state=42, early_stopping=True)
grid_mlp = GridSearchCV(estimator=mlp, param_grid=param_grid_mlp, cv=3, scoring='r2', verbose=1, n_jobs=-1)
grid_result_mlp = grid_mlp.fit(X_train, y_train)

# Melhor modelo e avaliação
best_model_mlp = grid_result_mlp.best_estimator_
y_pred_mlp = best_model_mlp.predict(X_test)

# Avaliação do modelo
print("Melhores parâmetros MLP:", grid_result_mlp.best_params_)
print("R² MLP no teste:", r2_score(y_test, y_pred_mlp))
print("MSE MLP no teste:", mean_squared_error(y_test, y_pred_mlp))


Fitting 3 folds for each of 360 candidates, totalling 1080 fits
Melhores parâmetros MLP: {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (256,), 'max_iter': 1000, 'solver': 'lbfgs'}
R² MLP no teste: -1.6680635152021934
MSE MLP no teste: 0.006020512608816839


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Carregar dados
data = pd.read_excel('C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx', sheet_name='Planilha1')
data['Data Referência'] = pd.to_datetime(data['Data Referência'])
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()

X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']


# Divisão dos dados
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Definir hiperparâmetros para GridSearch
param_grid_mlp = {
    'hidden_layer_sizes': [(64,), (128,), (64, 32)],
    'max_iter': [2000, 5000],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01],
    'activation': ['relu', 'tanh']
}

# Criar modelo e aplicar GridSearch
mlp = MLPRegressor(random_state=42, early_stopping=False)
grid_mlp = GridSearchCV(estimator=mlp, param_grid=param_grid_mlp, cv=3, scoring='r2', verbose=1, n_jobs=-1)
grid_result_mlp = grid_mlp.fit(X_train, y_train)

# Melhor modelo e avaliação
best_model_mlp = grid_result_mlp.best_estimator_
y_pred_mlp = best_model_mlp.predict(X_test)

# Avaliação do modelo
print("Melhores parâmetros MLP:", grid_result_mlp.best_params_)
print("R² MLP no teste:", r2_score(y_test, y_pred_mlp))
print("MSE MLP no teste:", mean_squared_error(y_test, y_pred_mlp))


Fitting 3 folds for each of 72 candidates, totalling 216 fits
Melhores parâmetros MLP: {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (64,), 'max_iter': 2000, 'solver': 'lbfgs'}
R² MLP no teste: -0.4613376658226258
MSE MLP no teste: 0.0032975233882906086


In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Carregar dados
data = pd.read_excel('C:\\Users\\JoãoEmileXimenesMuri\\Documents\\pessoal\\TCC\\Bases TCC\\Dados_TCC_2014_2024_Normalizados_TabUnica.xlsx', sheet_name='Planilha1')
data['Data Referência'] = pd.to_datetime(data['Data Referência'])
data['IPCA Próximo Mês'] = data['Valor IPCA'].shift(-1)
data = data.dropna()

X = data.drop(columns=['Data Referência', 'IPCA Próximo Mês'])
y = data['IPCA Próximo Mês']

# Divisão dos dados
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Definir hiperparâmetros para GridSearch
param_grid_mlp = {
    'hidden_layer_sizes': [(64,), (128,), (256,), (64, 64), (128, 64)],
    'max_iter': [1000, 2000, 5000],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001, 0.01],
    'activation': ['relu', 'tanh'],
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': [0.001, 0.01],
    'early_stopping': [True, False]
}

# Criar modelo e aplicar GridSearch
mlp = MLPRegressor(random_state=42)
grid_mlp = GridSearchCV(estimator=mlp, param_grid=param_grid_mlp, cv=3, scoring='r2', verbose=1, n_jobs=-1)
grid_result_mlp = grid_mlp.fit(X_train, y_train)

# Melhor modelo e avaliação
best_model_mlp = grid_result_mlp.best_estimator_
y_pred_mlp = best_model_mlp.predict(X_test)

# Avaliação do modelo
print("Melhores parâmetros MLP:", grid_result_mlp.best_params_)
print("R² MLP no teste:", r2_score(y_test, y_pred_mlp))
print("MSE MLP no teste:", mean_squared_error(y_test, y_pred_mlp))


Fitting 3 folds for each of 1440 candidates, totalling 4320 fits
Melhores parâmetros MLP: {'activation': 'relu', 'alpha': 0.01, 'early_stopping': True, 'hidden_layer_sizes': (256,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 1000, 'solver': 'lbfgs'}
R² MLP no teste: -1.6680635152021934
MSE MLP no teste: 0.006020512608816839
